In [13]:
! bzip2 -dk 'data/simplewiki-20190220-pages-articles-multistream.xml.bz2'

bzip2: Output file data/simplewiki-20190220-pages-articles-multistream.xml already exists.


In [1]:
from bs4 import BeautifulSoup

In [18]:
def load_pages(file_name):
    file_name = 'data/simplewiki-20190220-pages-articles-multistream.xml'
    in_page = False
    page_lines = None

    with open (file_name) as f:
        for line in f:
            if line.strip() == '<page>':
                in_page = True
                page_lines = []
            if in_page: 
                page_lines.append(line)
            if line.strip() == '</page>':
                in_page = False
        yield ''.join(page_lines)

In [20]:
load_pages('data/simplewiki-20190220-pages-articles-multistream.xml')

<generator object load_pages at 0x113bb3d00>

In [21]:
def load_page_details(file_name='data/simplewiki-20190220-pages-articles-multistream.xml'):
    for page in load_pages(file_name):
        soup = BeautifulSoup(page)
        output = {}
        output['title'] = soup.select_one('title').text
        texts = soup.select('text')
        if len(texts) > 0:
            output['text'] = texts[0]
        yield output

In [22]:
next(iter(load_page_details()))

{'title': 'Pecados y milagros',
 'text': <text xml:space="preserve">#REDIRECT [[Pecados y Milagros]]</text>}

In [23]:
import pymongo

In [26]:
from pymongo import MongoClient

mc = pymongo.MongoClient()
db = mc['wikicache']
simple_articles = db['simple_articles']

In [27]:
simple_articles.find_one()
